<a href="https://colab.research.google.com/github/GoDummyGo/GoDummyGo/blob/main/guide_for_beginners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[# Finlab API document](https://ai.finlab.tw/document)



# Install package

In [ ]:
%%capture
!pip install finlab > log.txt
!pip install talib-binary > log.txt

# Get data




In [ ]:
from finlab import data

pb = data.get('price_earning_ratio:股價淨值比')
close = data.get('price:收盤價')
close

輸入成功!


INFO:numexpr.utils:NumExpr defaulting to 2 threads.


,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,0059,0060,0061,006201,006202,006203,006204,006205,006206,006207,006208,00625K,00631L,00632R,00633L,00634R,00635U,00636,00636K,00637L,00638R,00639,00640L,00641R,00642U,00643,00643K,00645,00646,00647L,...,9908,9910,9911,9912,9914,9915,9917,9918,9919,9921,9922,9924,9925,9926,9927,9928,9929,9930,9931,9933,9934,9935,9937,9938,9939,9940,9941,9941A,9942,9943,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2007-04-23,9.54,57.85,32.83,38.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.60,28.00,10.35,14.00,48.55,32.35,56.5,21.40,16.55,52.0,1.87,29.35,31.05,16.60,34.60,18.30,4.80,37.00,15.80,28.55,48.55,9.93,27.55,31.25,30.80,95.00,21.60,NaN,48.70,19.70,31.0,30.35,9.78,73.60,13.25,51.3,72.40,NaN,46.00,49.60
2007-04-24,9.54,58.10,32.99,38.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.50,27.75,10.35,14.40,48.95,32.60,55.5,21.30,16.40,52.5,1.83,29.30,30.95,16.45,34.80,18.25,4.82,39.55,15.30,28.30,47.80,9.75,27.30,31.40,30.60,94.50,21.50,NaN,48.70,19.65,32.0,30.50,9.91,75.00,13.25,50.5,71.60,NaN,45.90,50.40
2007-04-25,9.52,57.60,32.80,38.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.10,27.80,10.20,14.00,49.75,32.10,55.5,21.20,17.35,54.1,1.83,29.15,30.70,16.40,34.95,18.15,4.71,42.30,16.20,28.10,46.80,9.68,28.35,31.30,30.00,94.40,21.55,NaN,48.60,19.65,32.3,29.50,10.10,74.60,13.30,49.9,71.60,NaN,49.10,49.10
2007-04-26,9.59,57.70,32.80,38.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.25,27.90,10.05,14.00,49.60,32.50,57.4,21.05,16.95,55.4,1.90,29.20,30.70,16.40,34.35,18.05,4.81,42.30,15.80,28.00,46.80,9.81,28.20,31.45,29.75,94.90,21.65,NaN,47.65,19.50,31.6,29.15,10.80,74.50,13.25,49.5,71.00,NaN,48.90,48.00
2007-04-27,9.55,57.50,32.72,38.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.25,27.75,9.90,14.00,49.00,31.65,56.8,21.00,17.20,55.6,1.77,29.35,30.70,16.60,34.05,18.00,4.85,40.00,15.70,27.80,44.70,10.20,28.20,29.50,29.35,95.00,21.60,NaN,47.80,19.35,31.4,28.50,11.55,75.70,13.15,48.8,69.50,NaN,48.60,46.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,NaN,136.40,56.80,124.10,66.20,30.84,23.90,30.86,93.45,NaN,NaN,NaN,22.99,21.00,NaN,64.75,84.80,37.55,36.65,30.20,78.30,NaN,125.30,5.57,58.50,3.88,23.96,25.80,9.25,23.28,7.96,16.72,30.12,10.14,14.72,18.84,NaN,26.27,38.86,72.55,...,33.00,214.00,70.00,8.65,271.00,NaN,99.5,43.60,22.10,317.0,NaN,41.15,38.40,48.15,32.70,6.10,9.42,45.50,39.50,36.15,15.25,19.70,48.20,82.90,65.60,34.45,162.50,51.1,87.80,61.30,21.7,61.90,15.35,11.55,10.30,82.0,16.75,105.5,27.20,16.40
2021-10-26,NaN,137.55,57.55,125.10,66.65,31.20,24.06,31.05,93.90,NaN,NaN,NaN,23.13,21.20,NaN,65.50,84.85,37.69,36.69,30.33,79.00,NaN,127.55,5.52,58.60,3.86,24.03,25.79,9.26,23.31,7.95,16.77,30.87,9.99,14.58,18.98,NaN,26.59,39.12,73.45,...,33.10,218.50,70.00,8.75,278.50,NaN,100.0,43.85,22.10,325.0,NaN,41.20,38.60,48.30,32.75,6.12,9.49,45.60,40.25,36.40,15.30,20.05,48.30,84.20,65.30,33.95,167.50,51.1,87.90,61.00,21.8,67.20,15.35,11.70,9.87,82.7,16.75,106.0,27.50,16.30
2021-10-27,NaN,137.55,57.70,125.75,66.65,31.19,24.24,31.15,94.00,NaN,NaN,NaN,22.78,21.60,NaN,65.50,85.25,37.12,35.99,29.83,79.00,NaN,12

# Write startegy position

### strategy condition
價格 > 20 日均線入場, 價格 < 60 日均線出場，最多持有10檔，超過 10 個進場訊號，則以股價淨值比小的股票優先選入。

## hold_until function
這大概是所有策略撰寫中，最重要的語法糖，上述語法中 entries 為進場訊號，而 exits 是出場訊號。所以 entries.hold_until(exits) ，就是進場訊號為 True 時，買入並持有該檔股票，直到出場訊號為 True 則賣出。

Screen-Shot-2021-10-26-at-6-35-05-AM
此函式有很多細部設定，可以讓你最多選擇 N 檔股票做輪動。另外，當超過 N 檔進場訊號發生，也可以按照客制化的排序，選擇優先選入的股票。最後，可以持外設定停損停利，來增加出場的時機點。以下是 hold_until 的參數說明：

*   exit (pd.Dataframe): 出場訊號。
*   nstocks_limit (int): 持有檔數上限。
*   stoploss (float): 停損基準。範例：0.1，代表成本價下跌 10% 時出場。
*   takeprofit (float): 停利基準。範例：0.1，代表成本價上漲 10% 時出場。
*   trade_at (str): 停損停利參考價。可選 close 或 open。
*   rank (pd.Dataframe): 當天進場訊號超過 nstocks_limit 時，以 rank 數字越大的股票優先進場。

In [ ]:
from finlab import data
from finlab.backtest import sim

close = data.get('price:收盤價')
pb = data.get('price_earning_ratio:股價淨值比')  

sma20 = close.average(20)
sma60 = close.average(60)

entries = close > sma20
exits = close < sma60

position = entries.hold_until(exits, nstocks_limit=10, rank=-pb, takeprofit=0.4)

# Backtest

## Display strategy result

In [ ]:
from finlab.backtest import sim

backtest_report = sim(position,upload=False)
backtest_report.display()

1262    0.1
1416    0.1
2362    0.1
2374    0.1
2897    0.1
2936    0.1
3489    0.1
4432    0.1
4552    0.1
8443    0.1
Name: 2021-10-29 00:00:00, dtype: float64

Timestamp('2021-10-29 00:00:00')

## Get startegy returns time series

In [ ]:
backtest_report.creturn

date
2007-05-10    1.000000
2007-05-11    1.002896
2007-05-14    1.012405
2007-05-15    0.993535
2007-05-16    0.989059
                ...   
2021-10-25    9.442165
2021-10-26    9.596957
2021-10-27    9.614560
2021-10-28    9.685518
2021-10-29    9.679848
Length: 3574, dtype: float64

## Get trade record

##### trade_record_df重點欄位說明:
* exit_sig_date:進場訊號產生日。
* entry_sig_date:出場訊號產生日。
* entry_date:進場日。
* exit_date:出場日。
* period:持有天數。
* position: 持有佔比。
* return:報酬率。
* mdd:持有期間最大回撤。
* mae_return:持有期間最大不利幅度。
* g_mfe_return:持有期間最大有利幅度。
* b_mfe_return:mae_return發生前的最大有利幅度。
* atr_return:標準化後ATR。
* edge_ratio:優勢比率，g_mfe_return/mae_return。



In [ ]:
trade_record, _ = backtest_report.get_trade_record()
trade_record

,exit_sig_date,entry_sig_date,stock_id,mdd,position,period,atr,mae_price,b_mfe_price,g_mfe_price,exit_price,entry_price,return,mae_return,b_mfe_return,g_mfe_return,atr_return,edge_ratio,entry_date,exit_date,market,category
0,2007-11-16,2007-10-12,1101 台泥,22.01,0.1,26,2.471435,46.826393,60.039257,60.039257,46.826393,57.713793,-19.37,19.37,3.43,3.43,4.28,0.177078,2007-10-15,2007-11-19,sii,水泥工業
1,2010-09-30,2010-07-05,1103 嘉泥,10.39,0.1,63,0.292843,15.580899,19.189341,19.189341,17.424150,15.580899,11.19,0.58,22.47,22.47,1.88,38.741379,2010-07-06,2010-10-01,sii,水泥工業
2,2011-06-21,2011-06-16,1103 嘉泥,2.51,0.1,4,0.379611,19.929583,20.442058,20.442058,19.929583,20.442058,-3.08,3.08,0.00,0.00,1.86,0.000000,2011-06-17,2011-06-22,sii,水泥工業
3,2011-08-04,2011-07-12,1103 嘉泥,12.98,0.1,18,0.728848,20.116461,23.118316,23.118316,20.116461,21.865600,-8.55,8.55,5.12,5.12,3.33,0.598830,2011-07-13,2011-08-05,sii,水泥工業
4,2014-05-13,2014-04-11,1103 嘉泥,7.12,0.1,22,0.328831,19.472114,20.965183,20.965183,19.472114,19.969804,-3.07,3.07,4.38,4.38,1.65,1.426710,2014-04-14,2014-05-14,sii,水泥工業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,2015-07-27,2015-07-23,9958 世紀鋼,2.03,0.1,3,0.246700,11.733493,11.977139,11.977139,11.759140,11.977139,-2.40,2.61,0.00,0.00,2.06,0.000000,2015-07-24,2015-07-28,sii,鋼鐵工業
1793,2007-06-04,2007-05-08,9962 有益,6.92,0.1,20,1.035714,45.050000,48.400000,48.400000,45.650000,48.400000,-6.24,7.48,0.00,0.00,2.14,0.000000,2007-05-09,2007-06-05,otc,鋼鐵工業
1794,2009-08-18,2009-06-18,9962 有益,10.96,0.1,43,0.644689,12.692308,13.802885,15.442308,13.855769,13.802885,-0.20,8.60,0.00,11.24,4.67,1.306977,2009-06-19,2009-08-19,otc,鋼鐵工業
1795,2009-11-27,2009-11-24,9962 有益,1.47,0.1,4,0.430632,14.173077,14.384615,14.384615,14.173077,14.384615,-2.05,2.05,0.00,0.00,2.99,0.000000,2009-11-25,2009-11-30,otc,鋼鐵工業


In [ ]:
import plotly.express as px

trade_record['profit_loss'] = trade_record['return'].apply(lambda s: 'profit' if s > 0 else 'loss')
fig = px.histogram(trade_record, x="return", color="profit_loss", title='profit_loss_hist')

return_mean = round(trade_record['return'].mean(), 2)
win_ratio = round(sum(trade_record['profit_loss'] == 'profit') / len(trade_record['profit_loss']) * 100, 2)
fig.add_vline(x=return_mean, line_width=3, line_dash="dash", line_color="green",
              annotation_position="top left",
              annotation_text=f'avg_return:{return_mean}', row=1, col=1)
fig.add_vline(x=win_ratio, line_width=3, line_dash="dash", line_color="purple",
              annotation_position="top right",
              annotation_text=f'win_ratio:{win_ratio}', row=1, col=1)

## Upload strategy

sim中的upload預設為true，會顯示網頁模式的結果。

複製下列cell的內容貼到finlab gui，才可設定每日自動更新策略清單。

[細節見文件"策略管理介面"。](https://ai.finlab.tw/document)

指標解析:

夏普值小於零，代表賠錢的意思，以一個簡易的選股策略來說，一般人波段實單操作，可能是0.7，以一個ETF來說，可能 0.9 就已經很不錯了，而以網路上付費選股策略 1.3 可能會是比較理想的數值，當然也有真的很厲害的選股策略，Sharpe可以到 2 或 3。

獲利峰態是跟一般的高斯分佈做比較，比高斯分佈更尖 kurt 越高，通常代表策略績效越穩定，而一般股票而言大盤的kurt大約在 1~2 左右，大於 2 基本上算是勉強堪用，5以上就算還不錯。偏態的話也是跟高斯分佈比較，越偏向右邊越小，通常 -0.5 以下算是勉強堪用，-1左右算是還不錯

In [ ]:
from finlab import data
from finlab.backtest import sim

close = data.get('price:收盤價')
pb = data.get('price_earning_ratio:股價淨值比')  

sma20 = close.average(20)
sma60 = close.average(60)

entries = close > sma20
exits = close < sma60

position = entries.hold_until(exits, nstocks_limit=10, rank=-pb, takeprofit=0.4)
sim(position, name='策略教學範例:pb_ma')


# Other strategy demo

## 指定日期換股

position的dataframe用營收的index，不要用收盤價的index。
resample設D，就會是每月營收截止日換股（不一定是10日，遇假日可能往後），resample設D會依照position逐筆判斷進場訊號。

[PEG策略教學](https://www.finlab.tw/finlab-tw-stock-peg-strategy/)

In [ ]:
from finlab import data
from finlab.backtest import sim

pe = data.get('price_earning_ratio:本益比')
rev=data.get('monthly_revenue:當月營收')
rev_ma3=rev.average(3)
rev_ma12=rev.average(12)
營業利益成長率=data.get('fundamental_features:營業利益成長率')
peg=(pe/營業利益成長率)
cond1= rev_ma3/rev_ma12>1.1
cond2=rev/rev.shift(1)>0.9

cond_all=cond1 & cond2
result=peg*(cond_all)

# 營收截止日換股
position=result[result>0].is_smallest(10).reindex(rev.index_str_to_date().index, method='ffill')

sim(position=position,name="策略教學範例:peg_rev",stop_loss=0.1,upload=False)


## 做空回測

position (pd.Dataframe): 買賣訊號紀錄。 True 為持有， False 為空手。 若選擇做空-position，只要將 sim(position) 改成負的 sim(-position.astype(float))即可做空。 

In [ ]:
from finlab import data
from finlab.backtest import sim

close = data.get('price:收盤價')
pb = data.get('price_earning_ratio:股價淨值比')  

sma20 = close.average(20)
sma60 = close.average(60)

entries = close > sma20
exits = close < sma60

position = entries.hold_until(exits, nstocks_limit=10, rank=-pb)
sim(-position, name='策略教學範例:',upload=False).display()

1337   -0.1
1416   -0.1
2888   -0.1
2897   -0.1
3489   -0.1
4141   -0.1
4432   -0.1
4552   -0.1
5533   -0.1
8443   -0.1
Name: 2021-10-29 00:00:00, dtype: float64

Timestamp('2021-10-29 00:00:00')

## 技術指標選股

高 RSI 技術指標策略

上述策略會選出很多檔標的，要是我們只想要選出 20 檔要怎麼寫呢？這個策略我們可以將 RSI 最大的 20 檔股票納入組合，並且持有一週，來試試看效果如何。我們可以用 data.indicator 來計算所有股票的 RSI，也可以用 rsi.is_largest 來計算此股票的 RSI 是否是最大的 20 檔。



In [ ]:
from finlab import data
from finlab.backtest import sim

# 選出 RSI 最大的 20 檔股票
rsi = data.indicator('RSI')
position = rsi.is_largest(20)

# 回測，每週(W)調整一次
report = sim(position, resample='W', name="高RSI策略", upload=False)